In [1]:

%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,pds,pdp,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                    d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,pUpdy_double,U, S_val, Sz_val, AorB_sym): 
    diff = round(abs(epx-epy)*0.5,3)
    if Norb==7:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp) \
                  +'_A'+str(A)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Vpp'+str(Vpp)+'_Upd'+str(Upd) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp)+'_diff'+str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)+\
                '_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                   +'_A'+str(B)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Vpp'+str(Vpp)+'_Upd'+str(Upd) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_pps'+str(pps)+'_ppp'+str(ppp)+'_diff'+str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                   +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta/4.0)
   
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpddiff,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,pds,pdp,pps,ppp)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    Esite  = ham.create_edep_diag_matrix(VS,A,epx,epy)
    
    H0 = T_pd + T_pp + Esite  
    
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_new = U_d.dot(H0.dot(U))
    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint = ham.create_interaction_matrix_ALL_syms(VS,d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,\
                                                      pUpdy_double,S_val, Sz_val,AorB_sym,A,Upp,Udif,Vpp,Upd,Updf)
        if pam.if_H0_rotate_byU==1:
            H = H0_new + Hint 
        else:
            H = H0 + Hint 
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs, wgt_d8, wgt_d9L, wgt_d10L2 = gs.get_ground_state(H, VS, S_val, Sz_val)
            if Norb==7:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,epx,epy,tpd,vals[0])
                util.write_GS_components('GS_weights_'+flowpeak+'.txt',A,epx,epy,tpd,wgt_d8, wgt_d9L, wgt_d10L2)
            elif Norb==9 or Norb==10 or Norb==11:
                util.write_GS2('Egs_'+flowpeak+'.txt',A,epx,epy,pds,pdp,vals[0])
                util.write_GS_components2('GS_weights_'+flowpeak+'.txt',A,epx,epy,pds,pdp,wgt_d8, wgt_d9L, wgt_d10L2)
            
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        # compute d8
#         fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)
        
        # compute d9L
#         b1L_state_indices, a1L_state_indices, b1L_state_labels, a1L_state_labels \
#                 = getstate.get_d9L_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, b1L_state_indices, b1L_state_labels, "Aw_b1L_", fname)
#         fig.compute_Aw1(H, VS, w_vals, a1L_state_indices, a1L_state_labels, "Aw_a1L_", fname)
        
#         # compute d10L2
#         d10L2_state_indices, d10L2_state_labels = getstate.get_d10L2_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, d10L2_state_indices, d10L2_state_labels, "Aw_d10L2_", fname)
        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    ed   = pam.ed

    As = pam.As
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    
    d_double,pUppx_double,pUppy_double,pUpp_double, pVpp_double, pUpdx_double,pUpdy_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
       
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)

    U_d = (U.conjugate()).transpose()
    # check if U if unitary
    #checkU_unitary(U,U_d)
    
    if Norb==7:
        
        for tpd in pam.tpds:
            for tpddiff in pam.tpddiffs:
                tpddiff = round(tpddiff,3)
                for ep1 in pam.ep_avgs: 

                    for ep2 in pam.ep_difs: 

                        epx = ep1+ep2
                        epy = ep1-ep2

                        for A in pam.As:
                            util.get_atomic_d8_energy(A,B,C)
                            for tpp in pam.tpps:
                                for Upp in pam.Upps:
                                    Upp = round(Upp,3)
                                    for Udif in pam.Udifs:
                                        Udif = round(Udif,3)
                                        for Vpp in pam.Vpps:
                                            Vpp = round(Vpp,3)

                                            for Upd in pam.Upds:
                                                Upd = round(Upd,3)

                                                for Updf in pam.Updfs:

                                                    Updf = round(Updf,3) 
                                                    print ('===================================================')
                                                    print ('A=',A, 'epx=', epx, 'epy=', epy, ' tpd=',tpd,' tpddiff=',tpddiff,' tpp=',tpp, \
                                                           ' Upp=',Upp,'Udif=',Udif,'Vpp=',Vpp,'Upd=',Upd,'Updf=',Updf)
                                                    compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,0,0,0,0,Upp,Udif,Vpp,Upd,Updf,\
                                                                    d_double,pUppx_double,pUppy_double,pUpp_double,\
                                                                    pVpp_double,pUpdx_double,\
                                                                    pUpdy_double,U, S_val, Sz_val, AorB_sym)

                                
    elif Norb==9 or Norb==10 or Norb==11:
        pps = pam.pps
        ppp = pam.ppp
        for ii in range(0,len(pam.pdps)):
            pds = pam.pdss[ii]
            pdp = pam.pdps[ii]
            for ep1 in pam.ep_avgs: 
                for ep2 in pam.ep_difs: 
                    epx = ep1+ep2
                    epy = ep1-ep2
                    for A in pam.As:
                        util.get_atomic_d8_energy(A,B,C)
                        for Upp in pam.Upps:
                            
                            Upp = round(Upp,3)
                            for Udif in pam.Udifs:
                                Udif = round(Udif,3)
                                for Vpp in pam.Vpps:
                                    Vpp = round(Vpp,3)
                                    for Upd in pam.Upds:
                                        Upd = round(Upd,3)
                                        for Updf in pam.Updfs:
                                            Updf = round(Updf,3)
                                            print ('===================================================')
                                            print ('A=',A, 'epx=', epx, 'epy=', epy,' pds=',pds,' pdp=',pdp,\
                                                   ' pps=',pps,' ppp=',ppp, ' Upp=',Upp, ' Udif=',Udif,' Vpp=',Vpp ,' Upd=',Upd,)
                                            compute_Aw_main(A,epx,epy,0,0,pds,pdp,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                                                            d_double,pUpp_double,pVpp_double,pUpdx_double,pUpdy_double,\
                                                            U, S_val, Sz_val, AorB_sym)   


    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px']
O2_orbs =  ['py']
compute A(w) for symmetries =  ['1A1', '1B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 10
VS.dim =  11025
No. of states with count_upup, count_updn, count_dnup, count_dndn: 0 5575 5450 0
len(d_list) 25
len(pUpp_list) 0
len(pUppx_list) 50
len(pUppy_list) 50
len(pVpp_list) 360
len(pUpdx_list) 20
len(pUpdy_list) 20
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 3.5 epy= 3.5  tpd= 1.5  tpddiff= -0.2  tpp= 0.55  Upp= 0.0 Udif= 0.0 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.33596572 -0.73005456 -0.63876667 -0.61883146 -0.59613303 -0.59613303
 -0.59613303 -0.59613303 -0.47300055 -0.46039568]
eige

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.34291416 -0.73197225 -0.64120883 -0.62204584 -0.59913127 -0.59913127
 -0.59913127 -0.47593929 -0.46338512 -0.39412541]
eigenvalue =  -2.3429141595587124
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.008101044350354313
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.027747876015144404
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.022510670739790756
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.00495045041923559
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004950450419235588
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.0067022167028727275
no e-h state  2900 d3z2r2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.003092089766183843
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.11049880928

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.35031838 -0.7342109  -0.64414721 -0.62582263 -0.60268464 -0.60268464
 -0.60268464 -0.60268464 -0.47945374 -0.46695666]
eigenvalue =  -2.350318384796262
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.00784026216878902
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.02759165591165385
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.02301390918937098
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004932118863989711
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004932118863989705
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.006631525033322114
no e-h state  2900 d3z2r2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.0031766327516526594
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.10836372079004

start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.35817521 -0.7367766  -0.64757529 -0.63016014 -0.60679235 -0.60679235
 -0.60679235 -0.60679235 -0.48354373 -0.47110915]
eigenvalue =  -2.358175208338262
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.007586863251511237
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.027431040740483154
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.023521766835273374
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004912433775505511
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004912433775505511
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.00655981215936434
no e-h state  2900 d3z2r2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.003258566822196179
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.10625018886263737
no e-h state  2912 dx2

sumweight= 0.9906301819151476
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 7.01 epy= 6.99  tpd= 1.5  tpddiff= -0.2  tpp= 0.55  Upp= 0.0 Udif= 0.0 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.18408904 1.65964795 2.26250964 2.4114571  2.67012964 2.90429262
 3.30259798 3.35336801 3.49885551 3.64501965]
eigenvalue =  1.1840890435501026
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.003152753279218513
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.012271886815082721
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.01044052258617948
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.0027327127312804403
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.0027327127312804476
no e-h state  2892 dx2y2 -

...

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.18611004 1.66026689 2.26397148 2.41181914 2.67084168 2.9058072
 3.30398639 3.35428134 3.50014554 3.64556131]
eigenvalue =  1.1861100365345136
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.010374404005464426
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.012292744356259225
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.003182787949479562
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004105401856061207
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004105401856061215
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.0027368062274858987
no e-h state  2900 d3z2r2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.0028446423091121257
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.18403702482673706
no 

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.19197316 1.66203585 2.26826898 2.41279849 2.67284273 2.91021018
 3.30798902 3.35684655 3.50382448 3.64687032]
eigenvalue =  1.1919731638702256
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.010177313409380419
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.01235467636297586
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.003274497217875089
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004137719486816402
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004137719486816387
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.0027488489085569228
no e-h state  2900 d3z2r2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.0026528194424314416
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.18230943190866486
no 

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.19753506 1.6636728  2.27243387 2.41361767 2.67464076 2.91439999
 3.31174633 3.35914872 3.50721493 3.64770554]
eigenvalue =  1.1975350632219297
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.009982142250311936
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.012415607712081233
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.003368647466986277
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.00416963360272501
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004169633602725012
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.002760523862416774
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.18056638074458345
no e-h state  2912 dx2y2 -1 2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.002760523862416781
no e-h

E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 7.09 epy= 6.91  tpd= 1.5  tpddiff= 0.2  tpp= 0.55  Upp= 0.0 Udif= 0.0 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.20279392 1.66517748 2.27646537 2.41427669 2.67623523 2.91837403
 3.31525645 3.36118684 3.51031473 3.64806736]
eigenvalue =  1.2027939152962874
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.009788918396123584
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.012475487359260734
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.0034652807289005383
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004201118407306037
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.004201118407306037
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , we

<Figure size 640x480 with 0 Axes>